In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
import os
import keras
from numpy.random import randint
from keras.optimizers import adam_v2
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import random
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
path_normal = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/test_normal/'
path_cap = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/test_cap/'
path_covid = '/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/test_covid/'

In [ ]:
normal_image = []
cap_image = []
covid_image = []
normal_files = os.listdir(path_normal)
cap_files = os.listdir(path_cap)
covid_files = os.listdir(path_covid)

In [ ]:
print('/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/test_normal/' + normal_files[0])
print(len(normal_files))
print(len(cap_files))
print(len(covid_files))

/content/drive/MyDrive/IndividualProject/SPGC-COVID/data/train_test/test_normal/normal067_114.jpg
3421
566
2843


In [ ]:
X = []
y = []
count = 0
for idx in range(450):
  file = normal_files[idx]
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_normal + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  X.append(img)
  y.append(0)

count = 0
for idx in range(450):
  file = cap_files[idx]
  count = count + 1
  if count % 200 == 0:
    print(count)
  temp = path_cap + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  X.append(img)
  y.append(1)
print(len(cap_image))

count = 0
for idx in range(450):
  count = count + 1
  file = covid_files[idx]
  if count % 200 == 0:
    print(count)
  temp = path_covid + file
  img = cv2.imread(temp)
  img = np.array(img)
  img = (img/255).astype("float32")
  X.append(img)
  y.append(2)
print(len(X), len(y))

200
400
200
400
0
200
400
1350 1350


In [ ]:
datagen1 = ImageDataGenerator(featurewise_center=True)
datagen1.fit(X)
mean = datagen1.mean
X = X-mean
print("mean value",mean)
mean = None
X = np.reshape(X, (len(X),512,512,3))
print(X.shape)
y = np.array(y)

mean value [[[0.04740741 0.04740741 0.04740741]]]
(1350, 512, 512, 3)


In [ ]:
model1 = Sequential([
    EfficientNetB5(include_top=False, weights='imagenet', input_shape=(512,512,3)),
    tf.keras.layers.GlobalAveragePooling2D(),
    Dense(1024,activation='tanh'),
    Dense(3, activation='softmax')
])

115277824/115263384 [==============================] - 1s 0us/step


In [ ]:
model1.load_weights("/content/drive/MyDrive/IndividualProject/SPGC-COVID/Results/0.3/three_class_EfficientNet_0.3/three_class_EfficientNet_0.3.ckpt")

In [ ]:
img = X[0]
img = np.reshape(img, (1, 512, 512, 3))
pred = model1(img)
print(pred[0])
print(np.argmax(pred[0]))

tf.Tensor([0.7686326  0.10741621 0.1239512 ], shape=(3,), dtype=float32)
0


In [ ]:
predictions = []
count = 0
for img in X:
  count = count + 1
  if count % 300 == 0:
    print(count)
  img = np.reshape(img, (1, 512, 512, 3))
  temp = model1(img)
  index = np.argmax(temp[0])
  predictions.append(index)
print(len(predictions))
print(predictions[0])

300
600
900
1200
1350
0


In [ ]:
print(predictions[0: 100])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
acc = sum(predictions == y) / len(predictions)
print("The total accuracy:" + str(acc))
normal_pred = predictions[: 450]
normal_y = y[: 450]
cap_pred = predictions[450: 900]
cap_y = y[450: 900]
covid_pred = predictions[900: ]
covid_y = y[900: ]
normal_acc = sum(normal_pred == normal_y) / len(normal_pred)
cap_acc = sum(cap_pred == cap_y) / len(cap_pred)
covid_acc = sum(covid_pred == covid_y) / len(covid_pred)
print("The normal cases accuracy:" + str(normal_acc))
print("The cap cases accuracy:" + str(cap_acc))
print("The covid cases accuracy:" + str(covid_acc))

The total accuracy:0.894074074074074
The normal cases accuracy:0.9688888888888889
The cap cases accuracy:0.9488888888888889
The covid cases accuracy:0.7644444444444445


In [ ]:
normal_normal = 0
normal_cap = 0
normal_covid = 0
cap_normal = 0
cap_cap = 0
cap_covid = 0
covid_normal = 0
covid_cap = 0
covid_covid = 0
for i in range(len(predictions)):
  if predictions[i] == 0:
    if y[i] == 0:
      normal_normal = normal_normal + 1
    elif y[i] == 1:
      normal_cap = normal_cap + 1
    else:
      normal_covid = normal_covid + 1
  elif predictions[i] == 1:
    if y[i] == 0:
      cap_normal = cap_normal + 1
    elif y[i] == 1:
      cap_cap = cap_cap + 1
    else:
      cap_covid = cap_covid + 1
  else:
    if y[i] == 0:
      covid_normal = covid_normal + 1
    elif y[i] == 1:
      covid_cap = covid_cap + 1
    else:
      covid_covid = covid_covid + 1

print("normal_normal:" + str(normal_normal))
print("normal_cap:" + str(normal_cap))
print("normal_covid:" + str(normal_covid))
print("cap_normal:" + str(cap_normal))
print("cap_cap:" + str(cap_cap))
print("cap_covid:" + str(cap_covid))
print("covid_normal:" + str(covid_normal))
print("covid_cap:" + str(covid_cap))
print("covid_covid:" + str(covid_covid))

normal_normal:436
normal_cap:20
normal_covid:94
cap_normal:4
cap_cap:427
cap_covid:12
covid_normal:10
covid_cap:3
covid_covid:344
